# Best places to open an Asian Fine-Dining place in London

<center>By Kartik Patil</center>

## Table of Contents
1. [Introduction](#introduction)
2. [Data](#data)
3. [Methodology](#methodology)
4. [Results and Discussion](#results)
5. [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The aim of the following report is to suggest locations around London which are ideal to open an Asian Fine-Dining Restaurants. An Asian fine-dining chain that serves Ramen, Pad Thai and other assorted dishes from East and South East Asia wants to make its foray into London and wants to find ideal locations for opening its restaurants. The idea behind having a successfull business is that the location should be able to attract a middle-class eating out crowd, while at the same time not facing intense competition from other restaurants, namely other fine-dining and asian restaurants. Therefore, the restaurants are opened based on the following premise:
* **The chain wants to open its restaurants in areas which do not already have a wide number of Asian restaurants**
* **The restaurants need to be as close to Central London due to ease of commute.**
* **The restaurants should be close to as possible to other rich areas, which have malls, business centres or tourists attractions, to attract the appropriate crowd.**

 

## Data <a name="data"></a>

To appropriately locate our restaurant and solve our business problem we need to find neighbourhoods which are:
1. **Not filled with Asian or Fine-Dining restaurants.**
2. **Are closer to Central London.**
3. **Are in expensive areas or areas with high amounts of movement.**
4. **Are as close to Central London as possible.**

The Data is gathered using the Foursquare API  to locate and categorise these restaurants into their respective categories. The data is then wrangled into a Pandas Dataframe for further analysis. We also use the to find the most visited areas in London and add that data to neighbourhoods. Furthermore, we encode------

In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


First we find the co-ordinates for the Centre of London, so as to calculate the distances of different neighbourhoods from the centre of London. In our case, we assume the centre of London to be Trafalgar Square and therefore, we find the co-ordinates for Trafalgar Square using the following code:

In [3]:
import requests

#define function to get coordinates using the Google Geocoding API
def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]  
address = 'Trafalgar Square, London, UK'
london_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, london_center))

Coordinate of Trafalgar Square, London, UK: [51.5075873, -0.1278296]


Next up we find all the neighborhoods in London as well as their latitudes and longitudes. We scrape this data off Wikipedia, using Beautiful Soup.

In [4]:
#retrieve London neighborhoods as beautiful soup objects
london_data = pd.DataFrame()
url = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
london_neigh = requests.get(url).text
soup = BeautifulSoup(london_neigh, 'html5lib')

In [5]:
#find and extract neighborhoods
table = soup.find('table', {'class':'wikitable sortable'}).tbody
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# Converts columns to pd dataframe
df = pd.DataFrame(columns = columns)
'''
Extracts every row with corresponding columns then appends the values to the create pd dataframe "df". The first row (row[0]) is skipped because it is already the header
'''
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)
        df

In [6]:
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [7]:
df.shape

(531, 6)

Once we have all the neighbourhoods, we get the latitudes and longitudes for each of the neighborhoods using the Google Maps geocoding API as well as the function we wrote above.

In [8]:
#get the latitudes and longitudes for all neighborhoods
lat_list = {}
for location in df['Location']:
    locale = location+", London, UK"
    lat_long=get_coordinates(google_api_key, locale)
    lat_list[location]={}
    lat_list[location]['latitude']=lat_long[0]
    lat_list[location]['longitude']=lat_long[1]
print("complete")

complete


In [9]:
#get the latitudes and longitudes in a dataframe
lat_list_df = pd.DataFrame()
for ind, location in enumerate(lat_list):
    latitude = lat_list[location]['latitude']
    longitude = lat_list[location]['longitude']
    lat_list_df=lat_list_df.append({"Location": location, 
                       "Latitude": latitude, 
                       "Longitude": longitude},
                        ignore_index=True)
lat_list_df.head()

,Latitude,Location,Longitude
0,51.492612,Abbey Wood,0.118818
1,51.508372,Acton,-0.274440
2,51.358673,Addington,-0.031254
3,51.380550,Addiscombe,-0.072274
4,51.426316,Albany Park,0.102809


We now clean the data to make it workable. Firstly, we do not need the Dial code or OS grid ref so we remove these two columns from our data frame. Furthermore, we remove the numbers next to London boroughs as these are the references from Wikipedia. Then we calculate the distance from centre for either of these neighborhoods and add the column to our dataframe.  Lastly, London is big and therefore, it isn't feasible to get venues in all neighbourhoods. Since, our client wants the restaurants to be situated closer to the centre we only keep those neighborhoods that are within 15 kms of Trafalgar Square. Once the data is all cleaned up we retrieve the venue details for each neighborhood using the Foursquare API.

In [12]:
#import geopy for distance
from geopy import distance
london_data = df.merge(lat_list_df, how='inner', on='Location')
london_data = london_data.drop(london_data.columns[[4,5]], axis=1)
london_data['London\xa0borough'] = london_data['London\xa0borough'].map(lambda x: x.rstrip('\]0123456789\['))
distance_list = {}
for location, lat, lng in zip(london_data['Location'], london_data['Latitude'], london_data['Longitude']):
    distance_list[location] = {}
    loc_lat_lng = (lat, lng)
    distance_list[location] = distance.distance(loc_lat_lng, london_center).km
distance_df = pd.DataFrame()
for ind, location in enumerate(distance_list):
    distance = distance_list[location]
    distance_df = distance_df.append({'Location':location, 'Distance from Center': distance}, ignore_index=True)
london_data = london_data.merge(distance_df, how='inner', on='Location')  
drop_index = london_data[london_data['Distance from Center'] > 15].index
london_data = london_data.drop(drop_index)
london_data.head()

,Location,London borough,Post town,Postcode district,Latitude,Longitude,Distance from Center
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.508372,-0.274440,10.179297
3,Addiscombe,Croydon,CROYDON,CR0,51.380550,-0.072274,14.652051
6,Aldgate,City,LONDON,EC3,51.513438,-0.077171,3.576673
7,Aldwych,Westminster,LONDON,WC2,51.513266,-0.117183,0.972334
8,Alperton,Brent,WEMBLEY,HA0,51.539601,-0.298837,12.391598


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print("complete")
    return(nearby_venues)

In [15]:
london_venues = getNearbyVenues(names=london_data['Location'],
                                   latitudes=london_data['Latitude'],
                                   longitudes=london_data['Longitude']
                                  )
london_venues.head()

complete


,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,51.508372,-0.27444,London Star Hotel,51.509624,-0.272456,Hotel
1,Acton,51.508372,-0.27444,The Aeronaut,51.508376,-0.275216,Pub
2,Acton,51.508372,-0.27444,Amigo's Peri Peri,51.508396,-0.274561,Fast Food Restaurant
3,Acton,51.508372,-0.27444,Dragonfly Brewery at George & Dragon,51.507378,-0.271702,Brewery
4,Acton,51.508372,-0.27444,North China Restaurant,51.508251,-0.277435,Chinese Restaurant


In [16]:
london_venues = london_venues.merge(distance_df, how='inner', on='Location')
london_venues.head()

,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance from Center
0,Acton,51.508372,-0.27444,London Star Hotel,51.509624,-0.272456,Hotel,10.179297
1,Acton,51.508372,-0.27444,The Aeronaut,51.508376,-0.275216,Pub,10.179297
2,Acton,51.508372,-0.27444,Amigo's Peri Peri,51.508396,-0.274561,Fast Food Restaurant,10.179297
3,Acton,51.508372,-0.27444,Dragonfly Brewery at George & Dragon,51.507378,-0.271702,Brewery,10.179297
4,Acton,51.508372,-0.27444,North China Restaurant,51.508251,-0.277435,Chinese Restaurant,10.179297


## Methodology <a name="methodology"></a>

We now use our use the Pandas dataframe to find the best locations to open a Asian fine dining restaurant. In order to do so, we first do one hot encoding to convert all venue categories in dummies and then use clustering to find the similar neighborhoods. We use the one hot encoding to find the most popular venues in each neighborhood and then cluster regions into 5 clusters. We then observe the clusters for similarity. 

In [17]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Location'] = london_venues['Location'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Location,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,College Science Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop

In [18]:
london_grouped = london_onehot.groupby('Location').mean().reset_index()

In [19]:
london_grouped.head()

,Location,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus School,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,College Science Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Housing Development,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
#top 10 most common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Location'] = london_grouped['Location']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Location,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Pub,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Grocery Store,Athletics & Sports,Supermarket,Turkish Restaurant,Coffee Shop,Fast Food Restaurant
1,Addiscombe,Park,Bakery,Grocery Store,Indian Restaurant,Tram Station,Chinese Restaurant,Café,Fast Food Restaurant,Cosmetics Shop,Historic Site
2,Aldgate,Hotel,Coffee Shop,Gym / Fitness Center,Pizza Place,Cocktail Bar,English Restaurant,Restaurant,Pub,Italian Restaurant,Japanese Restaurant
3,Aldwych,Theater,Pub,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Sandwich Place,Cocktail Bar,French Restaurant,Seafood Restaurant
4,Alperton,Indian Restaurant,Supermarket,Gym / Fitness Center,Bus Station,Asian Restaurant,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant


In [21]:
kclusters = 5

london_grouped_clustering = london_grouped.drop('Location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 3, 2, 1, 3, 0, 2, 3])

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_data

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Location'), on='Location')

london_merged.head() # check the last columns!

,Location,London borough,Post town,Postcode district,Latitude,Longitude,Distance from Center,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Acton,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.508372,-0.274440,10.179297,3,Pub,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Grocery Store,Athletics & Sports,Supermarket,Turkish Restaurant,Coffee Shop,Fast Food Restaurant
3,Addiscombe,Croydon,CROYDON,CR0,51.380550,-0.072274,14.652051,1,Park,Bakery,Grocery Store,Indian Restaurant,Tram Station,Chinese Restaurant,Café,Fast Food Restaurant,Cosmetics Shop,Historic Site
6,Aldgate,City,LONDON,EC3,51.513438,-0.077171,3.576673,3,Hotel,Coffee Shop,Gym / Fitness Center,Pizza Place,Cocktail Bar,English Restaurant,Restaurant,Pub,Italian Restaurant,Japanese Restaurant
7,Aldwych,Westminster,LONDON,WC2,51.513266,-0.117183,0.972334,3,Theater,Pub,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Sandwich Place,Cocktail Bar,French Restaurant,Seafood Restaurant
8,Alperton,Brent,WEMBLEY,HA0,51.539601,-0.298837,12.391598,2,Indian Restaurant,Supermarket,Gym / Fitness Center,Bus Station,Asian Restaurant,Falafel Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant


In [23]:
london_merged = london_merged.dropna(axis=0)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Location'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We now find the most common venues as well as reduce the distance from center. Since the number of venues is huge we only consider those venues which are less than 10km away from the city center. We observe that cluseter 3 is closer to hotesl and movie theatersand therefore more likely to attract a fine-dining crowd. Therefore, we selct those locations for our restaurant that are in cluster 3 and less than  10 kms away from the center. 

In [24]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]].head()

,London borough,Longitude,Distance from Center,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Ealing, Hammersmith and Fulham",-0.274440,10.179297,3,Pub,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Grocery Store,Athletics & Sports,Supermarket,Turkish Restaurant,Coffee Shop,Fast Food Restaurant
6,City,-0.077171,3.576673,3,Hotel,Coffee Shop,Gym / Fitness Center,Pizza Place,Cocktail Bar,English Restaurant,Restaurant,Pub,Italian Restaurant,Japanese Restaurant
7,Westminster,-0.117183,0.972334,3,Theater,Pub,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Sandwich Place,Cocktail Bar,French Restaurant,Seafood Restaurant
10,Islington,-0.104290,3.428213,3,Pub,Coffee Shop,Café,French Restaurant,Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Korean Restaurant,Indian Restaurant
16,Wandsworth,-0.149411,7.232908,3,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Supermarket,Bakery,Bar,Indian Restaurant,Sandwich Place,Steakhouse


In [25]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]].head()

,London borough,Longitude,Distance from Center,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Islington,-0.132381,6.383041,0,Pub,Grocery Store,Coffee Shop,Italian Restaurant,Café,Fast Food Restaurant,Indian Restaurant,Bike Rental / Bike Share,Seafood Restaurant,Sandwich Place
26,Islington,-0.118345,3.704740,0,Café,Grocery Store,Brewery,Coffee Shop,Ethiopian Restaurant,Train Station,Caucasian Restaurant,Escape Room,Nightclub,Bus Stop
27,Wandsworth,-0.165547,4.728954,0,Pub,Bus Stop,Park,Grocery Store,Thai Restaurant,Café,Mini Golf,Gym,Gym / Fitness Center,Bakery
43,Tower Hamlets,-0.048080,6.122627,0,Café,Pub,Harbor / Marina,Restaurant,Coffee Shop,Cocktail Bar,Park,Farmers Market,Food Truck,Middle Eastern Restaurant
51,Greenwich,0.018833,10.747825,0,Pub,Café,Grocery Store,Spa,Bus Stop,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant


In [26]:
restaurant_filtered = london_merged.loc[(london_merged['Distance from Center'] < 10)&(london_merged['Cluster Labels'] ==3)]

In [27]:
restaurant_filtered.head()

,Location,London borough,Post town,Postcode district,Latitude,Longitude,Distance from Center,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Aldgate,City,LONDON,EC3,51.513438,-0.077171,3.576673,3,Hotel,Coffee Shop,Gym / Fitness Center,Pizza Place,Cocktail Bar,English Restaurant,Restaurant,Pub,Italian Restaurant,Japanese Restaurant
7,Aldwych,Westminster,LONDON,WC2,51.513266,-0.117183,0.972334,3,Theater,Pub,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Sandwich Place,Cocktail Bar,French Restaurant,Seafood Restaurant
10,Angel,Islington,LONDON,"EC1, N1",51.534676,-0.104290,3.428213,3,Pub,Coffee Shop,Café,French Restaurant,Restaurant,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Korean Restaurant,Indian Restaurant
16,Balham,Wandsworth,LONDON,SW12,51.443989,-0.149411,7.232908,3,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Supermarket,Bakery,Bar,Indian Restaurant,Sandwich Place,Steakhouse
17,Bankside,Southwark,LONDON,SE1,51.508137,-0.095184,2.267391,3,Coffee Shop,Bar,Italian Restaurant,Seafood Restaurant,Café,Art Museum,Bakery,Wine Bar,Pub,Portuguese Restaurant


In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(restaurant_filtered['Latitude'], restaurant_filtered['Longitude'], restaurant_filtered['Location'], restaurant_filtered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion <a name="results"></a>

Our analysis shows that there are quite a few areas in London near the city center i.e. Trafalgar Square, which are in localities that are around hotels, theaters and tourist attractions and therefore capable of attracting a huge number of people looking to fine-dine. Furthermore, we find that most of the areas in cluster 3 which match the kind of neighborhoods we are looking for also lack other Asian fine-dining restaurants therefore, these are ideal to open branches of our restaurant. The map above shows these locations. 

We approached our problem by first gathering data about different neighborhoods and how far they are from the center of London. Then we filtered and cleaned our data and used the Foursquare API to collect data about venues and venue categories. We then use one hot encoding to get the dummies for each venue categories and finally we use kmeans clustering to cluster neighborhoods and find the most suitable areas for opening our locations and as mentioned above cluster 3 best suits our preferences. While these might not neccessary be the best locations for opening branches of our fine-dine restaurant, they surely exhibit potential. More nuanced analysis is needed to find further shortlist locations from those mentioned in the above map.

## Conclusion <a name="conclusion"></a>

The following project tried to find the best locations to establis branches for our fine-dine restaurant chain. We settled on cluster 3 neighborhoods that are less than 10km away from the city center. This should be the starting point for deciding the suitable locations for our restaurant and further analysis is needed to better shortlist locations.